<a href="https://colab.research.google.com/github/dansarmiento/ColaboratoryNotebooks/blob/main/Waterpolo_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# libraries for data analysis and graphing
import pandas as pd
import datetime
import numpy as np
import os
import sqlite3
import shutil
import hashlib

#create the database
waterpolo_stats = sqlite3.connect('waterpolo_stats')


In [2]:
# access the google sheets with all of the transcribed bench stats
# use the game tracker and data validation sheets as dimension tables

# https://docs.google.com/spreadsheets/d/1SS1olbn4zuh8VtErI_QjI2I_y-oSluAhLaeQWCIdozE/edit?gid=353770692#gid=353770692

df_sheet_id = '1SS1olbn4zuh8VtErI_QjI2I_y-oSluAhLaeQWCIdozE'

game_tracker_sheet = 'game_tracker'
data_validation_sheet = 'data_validation'


game_tracker = "https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}".format(df_sheet_id,game_tracker_sheet)
data_validation = "https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}".format(df_sheet_id,data_validation_sheet)



games = pd.read_csv(game_tracker)
players = pd.read_csv(data_validation, usecols=['players','player_id','last_name','first_name','grade','comp_group','practice_group'])

# write the dimension tables to the database
games.to_sql('games', waterpolo_stats, if_exists='replace', index=False)
players.to_sql('players', waterpolo_stats, if_exists='replace', index=False)


31

In [3]:
# iterate through all of the stats pages, and create a dataframe where the game_id and player_id are mapped
# then write to the database

# Define the list of sheets
game_sheet_list = [
    'BS_1001','BS_1002','BS_1003','BS_1004','BS_1007','BS_1008','BS_1009','BS_1010',
    'BS_1013','BS_1016','BS_1017','BS_1018','BS_1019','BS_1020','BS_1021','BS_1022',
    'BS_1023','BS_1024','BS_1025','BS_1026','BS_1027','BS_1029','BS_1032','BS_1033',
    'BS_1034','BS_1038','BS_1039','BS_1041','BS_1042','BS_1043','BS_1044','BS_1045',
    'BS_1051','BS_1053','BS_1054','BS_1055','BS_1056','BS_1057','BS_1058','BS_1060',
    'BS_1061','BS_1062','BS_1063', 'BS_1059'
]

# Function to process each sheet
def process_sheet(sheet_name):
    # Create the URL for the sheet
    sheet_url = "https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}".format(df_sheet_id, sheet_name)

    # Read the CSV data from the sheet
    game_data = pd.read_csv(sheet_url, dtype=object)

    # Rename columns
    game_data.columns = ['quarter', 'cap_num', 'action', 'game_id', 'cap', 'player',
                         'date', 'team', 'color', 'opponent', 'Game ID']

    # Fill missing data and clean up
    game_data['game_id'] = game_data['Game ID'].iloc[0]
    game_data['cap'] = game_data['cap'].fillna('1A')
    game_data['cap_num'] = game_data['cap_num'].fillna('1A')
    game_data = game_data.drop(columns=['date', 'team', 'color', 'opponent', 'Game ID'])

    # Merge with players DataFrame
    game_data = game_data.merge(players[['players', 'player_id']],
                                left_on='player',
                                right_on='players',
                                how='left')
    game_data = game_data.drop(columns=['players'])

    # Create caps DataFrame
    game_caps = game_data[['cap', 'player', 'player_id']].dropna(subset=['player'])
    game_caps['player_id'] = game_caps['player_id'].astype(int)

    # Drop unnecessary columns from the main DataFrame
    game_data = game_data.drop(columns=['cap', 'player', 'player_id'])

    # Merge with caps on the condition cap = cap_num
    game_data = game_data.merge(game_caps[['cap', 'player_id']],
                                left_on='cap_num',
                                right_on='cap',
                                how='left')
    game_data.drop(columns=['cap'], inplace=True)

    # Create a primary key column
    def create_hash_with_index(row):
        row_string = f"{row['quarter']}-{row['cap_num']}-{row['action']}-{row['game_id']}-{row['player_id']}-{row.name}"
        return hashlib.md5(row_string.encode()).hexdigest()

    game_data['primary_key'] = game_data.apply(create_hash_with_index, axis=1)

    # Append to SQL database
    game_data.to_sql('game_data', waterpolo_stats, if_exists='append', index=False)

# Loop through all sheets and process
for sheet in game_sheet_list:
    process_sheet(sheet)

In [4]:
# this is what the games tracker looks like on the database, there are nulls on the bench stats column where a stat page is not available
df = pd.read_sql_query('select * from games', waterpolo_stats)
df.head()

,date,team,opponent,game_id,context,bench stats
0,8/15/2024,Ayala JV Boys,Nogales,1001,None,1.0
1,8/23/2024,Ayala Varsity Boys,Don Lugo,1002,None,1.0
2,8/23/2024,Ayala JV Boys,Don Lugo,1003,None,1.0
3,8/30/2024,Ayala Frosh Soph Boys,Damien,1004,Ayala FS Tournament,1.0
4,8/31/2024,Ayala Frosh Soph Boys,None,1005,Ayala FS Tournament,NaN


In [5]:
# this is what the player dimension table looks like, it was lifted from the boys overall schedule
# only change made was giving them an index and putting Evan in the varsity competition group where he belongs
df1 = pd.read_sql_query('select * from players', waterpolo_stats)
df1.head()

,players,player_id,last_name,first_name,grade,comp_group,practice_group
0,"Arevalo, Diego",5001,Arevalo,Diego,10,JV,B
1,"Asher, Bhushan",5002,Asher,Bhushan,9,Frosh/Soph,B
2,"Baker, Luke",5003,Baker,Luke,10,Varsity,A
3,"Castro, Colby",5004,Castro,Colby,10,JV,A
4,"Chang, Evan",5005,Chang,Evan,12,Varsity,A


In [6]:
# evaluate if there are any unmapped rows
df2 = pd.read_sql_query('select * from game_data where player_id is null', waterpolo_stats)
df2.head()

,quarter,cap_num,action,game_id,player_id,primary_key


In [7]:
# make the report where I can see all columns
query = '''
select
g.quarter
, g.action
, g.game_id
, g.player_id
, p.players
, p.grade
, p.comp_group
, g2.date
, g2.opponent
from game_data as g
left outer join players as p
on g.player_id = p.player_id
left outer join games as g2
on g.game_id = g2.game_id
'''

df3 = pd.read_sql_query(query, waterpolo_stats)
df3.head()


,quarter,action,game_id,player_id,players,grade,comp_group,date,opponent
0,1,Block,1001,5019,"Oberlander, Landon",10,JV,8/15/2024,Nogales
1,2,Block,1001,5019,"Oberlander, Landon",10,JV,8/15/2024,Nogales
2,2,Block,1001,5019,"Oberlander, Landon",10,JV,8/15/2024,Nogales
3,2,Block,1001,5019,"Oberlander, Landon",10,JV,8/15/2024,Nogales
4,3,Block,1001,5019,"Oberlander, Landon",10,JV,8/15/2024,Nogales


In [8]:
view = '''
create view vw_game_details as
select
g.quarter
, g.action
, g.game_id
, g.player_id
, p.players
, p.grade
, p.comp_group
, g2.team
, g2.date
, g2.opponent
, g.primary_key
from game_data as g
left outer join players as p
on g.player_id = p.player_id
left outer join games as g2
on g.game_id = g2.game_id
'''
waterpolo_stats.execute(view)

In [9]:
query = '''
select * from vw_game_details

'''
report = pd.read_sql_query(query, waterpolo_stats)
report.head()

,quarter,action,game_id,player_id,players,grade,comp_group,team,date,opponent,primary_key
0,1,Block,1001,5019,"Oberlander, Landon",10,JV,Ayala JV Boys,8/15/2024,Nogales,c2b44918bebb838e533bbc8a96134624
1,2,Block,1001,5019,"Oberlander, Landon",10,JV,Ayala JV Boys,8/15/2024,Nogales,8b35265db185063dbd58a78da0375420
2,2,Block,1001,5019,"Oberlander, Landon",10,JV,Ayala JV Boys,8/15/2024,Nogales,e5016e630878e5774bf028f18caf4137
3,2,Block,1001,5019,"Oberlander, Landon",10,JV,Ayala JV Boys,8/15/2024,Nogales,d3b894640752d0388b759530fd6fb118
4,3,Block,1001,5019,"Oberlander, Landon",10,JV,Ayala JV Boys,8/15/2024,Nogales,108ff832f3816ddf3183f6a51b695556


In [10]:
# examine the aggregated view

view = '''
select
action
, count(action) as count
, game_id
, player_id
, players
, grade
, comp_group
, team
, date
, opponent
from vw_game_details
group by action, game_id, player_id, players, grade, comp_group, date, opponent, team
'''

report = pd.read_sql_query(view, waterpolo_stats)
report.to_csv('waterpolo_stats_abwp_2024.csv', index=False)

In [11]:
# aggregated view with action counts for visualization
view = '''
create view vw_game_stats as
select
action
, count(action) as count
, game_id
, player_id
, players
, grade
, comp_group
, team
, date
, opponent
from vw_game_details
group by action, game_id, player_id, players, grade, comp_group, date, opponent, team
'''
waterpolo_stats.execute(view)

In [12]:
# create the table style for the banquet packet
team_stats = '''
SELECT
    players AS "ATHLETE NAME",
    COUNT(DISTINCT game_id) AS "GAMES",
    SUM(CASE WHEN action = 'Shot' THEN count ELSE 0 END) AS "ATTEMPTS",
    SUM(CASE WHEN action = 'Goal' THEN count ELSE 0 END) AS "GOALS",
    SUM(CASE WHEN action = 'Assist' THEN count ELSE 0 END) AS "ASSISTS",
    SUM(CASE WHEN action = 'Steal' THEN count ELSE 0 END) AS "STEALS",
    SUM(CASE WHEN action = 'Ejection' THEN count ELSE 0 END) AS "EJECTIONS",
    SUM(CASE WHEN action = 'Cause Ejection' THEN count ELSE 0 END) AS "CAUSED EJECTIONS",
    SUM(CASE WHEN action = 'Turnover' THEN count ELSE 0 END) AS "TURNOVERS"
FROM
    vw_game_stats
GROUP BY
    players
ORDER BY
    "ATHLETE NAME";
    '''

team_stats = pd.read_sql_query(team_stats, waterpolo_stats)
team_stats.to_csv('team_stats.csv', index=False)

In [13]:
# create the table style for the banquet packet but with columns for goalies
block_stats = '''
SELECT
    players AS "ATHLETE NAME",
    COUNT(DISTINCT game_id) AS "GAMES",
    SUM(CASE WHEN action = 'Block' THEN count ELSE 0 END) AS "BLOCKS",
    SUM(CASE WHEN action = 'Shot'  THEN count ELSE 0 END) AS "ATTEMPTS",
    SUM(CASE WHEN action = 'Goal' THEN count ELSE 0 END) AS "GOALS",
    SUM(CASE WHEN action = 'Steal' THEN count ELSE 0 END) AS "STEALS",
    SUM(CASE WHEN action = 'Ejection' THEN count ELSE 0 END) AS "EJECTIONS",
    SUM(CASE WHEN action = 'Turnover' THEN count ELSE 0 END) AS "TURNOVERS"
FROM
    vw_game_stats
WHERE
    player_id in (5019,5020,5025)
GROUP BY
    players
ORDER BY
    "ATHLETE NAME";
    '''

goalie_stats = pd.read_sql_query(block_stats, waterpolo_stats)
goalie_stats.to_csv('goalie_stats.csv', index=False)

In [14]:
varsity_team_stats = '''
SELECT
    players AS "ATHLETE NAME",
    COUNT(DISTINCT game_id) AS "GAMES",
    SUM(CASE WHEN action = 'Shot' THEN count ELSE 0 END) AS "ATTEMPTS",
    SUM(CASE WHEN action = 'Goal' THEN count ELSE 0 END) AS "GOALS",
    SUM(CASE WHEN action = 'Assist' THEN count ELSE 0 END) AS "ASSISTS",
    SUM(CASE WHEN action = 'Steal' THEN count ELSE 0 END) AS "STEALS",
    SUM(CASE WHEN action = 'Ejection' THEN count ELSE 0 END) AS "EJECTIONS",
    SUM(CASE WHEN action = 'Cause Ejection' THEN count ELSE 0 END) AS "CAUSED EJECTIONS",
    SUM(CASE WHEN action = 'Turnover' THEN count ELSE 0 END) AS "TURNOVERS"
FROM
    vw_game_stats
WHERE
    comp_group = 'Varsity'
GROUP BY
    players
ORDER BY
    "ATHLETE NAME";
    '''

varsity_team_stats = pd.read_sql_query(varsity_team_stats, waterpolo_stats)
#varsity_team_stats.head()
varsity_team_stats.to_csv('varsity_team_stats.csv', index=False)

In [15]:
block_stats = '''
SELECT
    players AS "ATHLETE NAME",
    COUNT(DISTINCT game_id) AS "GAMES",
    SUM(CASE WHEN action = 'Block' THEN count ELSE 0 END) AS "BLOCKS",
    SUM(CASE WHEN action = 'Shot'  THEN count ELSE 0 END) AS "ATTEMPTS",
    SUM(CASE WHEN action = 'Goal' THEN count ELSE 0 END) AS "GOALS",
    SUM(CASE WHEN action = 'Steal' THEN count ELSE 0 END) AS "STEALS",
    SUM(CASE WHEN action = 'Ejection' THEN count ELSE 0 END) AS "EJECTIONS",
    SUM(CASE WHEN action = 'Turnover' THEN count ELSE 0 END) AS "TURNOVERS"
FROM
    vw_game_stats
WHERE
    player_id in (5025)
GROUP BY
    players
ORDER BY
    "ATHLETE NAME";
    '''

goalie_stats = pd.read_sql_query(block_stats, waterpolo_stats)
goalie_stats.to_csv('varsity_goalie_stats.csv', index=False)

In [16]:
block_stats = '''
SELECT
    players AS "ATHLETE NAME",
    COUNT(DISTINCT game_id) AS "GAMES",
    SUM(CASE WHEN action = 'Block' THEN count ELSE 0 END) AS "BLOCKS",
    SUM(CASE WHEN action = 'Shot'  THEN count ELSE 0 END) AS "ATTEMPTS",
    SUM(CASE WHEN action = 'Goal' THEN count ELSE 0 END) AS "GOALS",
    SUM(CASE WHEN action = 'Steal' THEN count ELSE 0 END) AS "STEALS",
    SUM(CASE WHEN action = 'Ejection' THEN count ELSE 0 END) AS "EJECTIONS",
    SUM(CASE WHEN action = 'Turnover' THEN count ELSE 0 END) AS "TURNOVERS"
FROM
    vw_game_stats
WHERE
    player_id in (5019)
GROUP BY
    players
ORDER BY
    "ATHLETE NAME";
    '''

goalie_stats = pd.read_sql_query(block_stats, waterpolo_stats)
goalie_stats.to_csv('JV_goalie_stats.csv', index=False)

In [17]:
block_stats = '''
SELECT
    players AS "ATHLETE NAME",
    COUNT(DISTINCT game_id) AS "GAMES",
    SUM(CASE WHEN action = 'Block' THEN count ELSE 0 END) AS "BLOCKS",
    SUM(CASE WHEN action = 'Shot'  THEN count ELSE 0 END) AS "ATTEMPTS",
    SUM(CASE WHEN action = 'Goal' THEN count ELSE 0 END) AS "GOALS",
    SUM(CASE WHEN action = 'Steal' THEN count ELSE 0 END) AS "STEALS",
    SUM(CASE WHEN action = 'Ejection' THEN count ELSE 0 END) AS "EJECTIONS",
    SUM(CASE WHEN action = 'Turnover' THEN count ELSE 0 END) AS "TURNOVERS"
FROM
    vw_game_stats
WHERE
    player_id in (5020)
GROUP BY
    players
ORDER BY
    "ATHLETE NAME";
    '''

goalie_stats = pd.read_sql_query(block_stats, waterpolo_stats)
goalie_stats.to_csv('frosh_soph_goalie_stats.csv', index=False)

In [18]:
pd.read_sql_query('select distinct comp_group from vw_game_stats', waterpolo_stats)

,comp_group
0,Frosh/Soph
1,JV
2,Varsity


In [19]:
varsity_team_stats = '''
SELECT
    players AS "ATHLETE NAME",
    COUNT(DISTINCT game_id) AS "GAMES",
    SUM(CASE WHEN action = 'Shot' THEN count ELSE 0 END) AS "ATTEMPTS",
    SUM(CASE WHEN action = 'Goal' THEN count ELSE 0 END) AS "GOALS",
    SUM(CASE WHEN action = 'Assist' THEN count ELSE 0 END) AS "ASSISTS",
    SUM(CASE WHEN action = 'Steal' THEN count ELSE 0 END) AS "STEALS",
    SUM(CASE WHEN action = 'Ejection' THEN count ELSE 0 END) AS "EJECTIONS",
    SUM(CASE WHEN action = 'Cause Ejection' THEN count ELSE 0 END) AS "CAUSED EJECTIONS",
    SUM(CASE WHEN action = 'Turnover' THEN count ELSE 0 END) AS "TURNOVERS"
FROM
    vw_game_stats
WHERE
    comp_group = 'JV'
GROUP BY
    players
ORDER BY
    "ATHLETE NAME";
    '''

varsity_team_stats = pd.read_sql_query(varsity_team_stats, waterpolo_stats)
#varsity_team_stats.head()
varsity_team_stats.to_csv('JV_team_stats.csv', index=False)

In [20]:
varsity_team_stats = '''
SELECT
    players AS "ATHLETE NAME",
    COUNT(DISTINCT game_id) AS "GAMES",
    SUM(CASE WHEN action = 'Shot' THEN count ELSE 0 END) AS "ATTEMPTS",
    SUM(CASE WHEN action = 'Goal' THEN count ELSE 0 END) AS "GOALS",
    SUM(CASE WHEN action = 'Assist' THEN count ELSE 0 END) AS "ASSISTS",
    SUM(CASE WHEN action = 'Steal' THEN count ELSE 0 END) AS "STEALS",
    SUM(CASE WHEN action = 'Ejection' THEN count ELSE 0 END) AS "EJECTIONS",
    SUM(CASE WHEN action = 'Cause Ejection' THEN count ELSE 0 END) AS "CAUSED EJECTIONS",
    SUM(CASE WHEN action = 'Turnover' THEN count ELSE 0 END) AS "TURNOVERS"
FROM
    vw_game_stats
WHERE
    comp_group = 'Frosh/Soph'
GROUP BY
    players
ORDER BY
    "ATHLETE NAME";
    '''

varsity_team_stats = pd.read_sql_query(varsity_team_stats, waterpolo_stats)
#varsity_team_stats.head()
varsity_team_stats.to_csv('Frosh_Soph_team_stats.csv', index=False)

In [21]:
df = pd.read_sql_query('select * from vw_game_stats', waterpolo_stats)
df.head()

,action,count,game_id,player_id,players,grade,comp_group,team,date,opponent
0,Assist,2,1003,5009,"Fox, Hudson",9,Frosh/Soph,Ayala JV Boys,8/23/2024,Don Lugo
1,Assist,1,1003,5012,"Huver, Liam",9,Frosh/Soph,Ayala JV Boys,8/23/2024,Don Lugo
2,Assist,1,1003,5031,"Wu, ZhengHong ""Mario""",12,JV,Ayala JV Boys,8/23/2024,Don Lugo
3,Assist,1,1004,5009,"Fox, Hudson",9,Frosh/Soph,Ayala Frosh Soph Boys,8/30/2024,Damien
4,Assist,1,1004,5014,"Khan, Sean",9,Frosh/Soph,Ayala Frosh Soph Boys,8/30/2024,Damien


In [22]:
df.action.unique()

array(['Assist', 'Block', 'Cause Ejection', 'Ejection', 'Goal', 'Rolled',
       'Shot', 'Steal', 'Turnover'], dtype=object)

In [23]:
df.to_sql('game_statistics', waterpolo_stats, if_exists='replace', index=False)

1328

In [24]:
# I need to insert rows into game statistics for zero for each action where the player_id is in the game
pd.read_sql_query('select * from game_statistics limit 5', waterpolo_stats)

,action,count,game_id,player_id,players,grade,comp_group,team,date,opponent
0,Assist,2,1003,5009,"Fox, Hudson",9,Frosh/Soph,Ayala JV Boys,8/23/2024,Don Lugo
1,Assist,1,1003,5012,"Huver, Liam",9,Frosh/Soph,Ayala JV Boys,8/23/2024,Don Lugo
2,Assist,1,1003,5031,"Wu, ZhengHong ""Mario""",12,JV,Ayala JV Boys,8/23/2024,Don Lugo
3,Assist,1,1004,5009,"Fox, Hudson",9,Frosh/Soph,Ayala Frosh Soph Boys,8/30/2024,Damien
4,Assist,1,1004,5014,"Khan, Sean",9,Frosh/Soph,Ayala Frosh Soph Boys,8/30/2024,Damien


In [25]:
df = pd.read_sql_query('select * from game_data', waterpolo_stats)
df.to_csv('game_data.csv', index=False)